<a href="https://colab.research.google.com/github/higebobo/my-colab/blob/main/pypf_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## pypfによるポイントアンドフィギュア算出

[pviglucci/pypf](https://github.com/pviglucci/pypf)

## インストールセクション

In [1]:
!pip install pypf yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## インポートセクション

In [2]:
from collections import OrderedDict
import datetime
from decimal import Decimal

from pypf.chart import PFChart
from pypf.instrument import Instrument
import yfinance as yf


## 定数定義

### 対象のシンボル(銘柄・指数・通貨ペア)

In [3]:
symbol_map = {
    # 'AMZN': 'Amazon.com Inc.',
    'GOOGL': 'Alphabet Inc.',
    # 'TSLA': 'Tesla Inc.',
    # 'MSFT': 'Microsoft Corporation',
    # 'AAPL': 'Apple Inc.',
    # 'DIS': 'Walt Disney Company',
    # '7203.T': 'トヨタ自動車',
    # '9101.T': '日本郵船',
    # '5401.T': '日本製鉄',
    # '2914.T': 'JT'
}

### 対象期間

In [4]:
# 本日まで
end = datetime.date.today()
start = end - datetime.timedelta(weeks=26*2)

### ポイントアンドフィギュアパラーメータ

In [5]:
# ボックスサイズ
box_size = 0.01

# データ取得期間(単位年)
period = 1

# メソッド c:終値 hl:高値安値
method = 'c'

# 枠反転
reversal = 3


## クラス定義

In [6]:
class NewYahooSecurity(Instrument):
    def __init__(self, symbol, force_cache=False):
        super().__init__(symbol=symbol, period=10)

    def populate_data(self):
        self.daily_historical_data = OrderedDict()

        end = datetime.date.today()
        start = end - datetime.timedelta(weeks=self.period*52)
        df = yf.download(symbol, start=start.strftime('%Y-%m-%d'), end=end.strftime('%Y-%m-%d'))
        q = Decimal('0.01')

        for index, row in df.iterrows():
            row['Date'] = index.strftime('%Y-%m-%d')
            row['Open'] = Decimal(row['Open']).quantize(q)
            row['High'] = Decimal(row['High']).quantize(q)
            row['Low'] = Decimal(row['Low']).quantize(q)
            row['Close'] = Decimal(row['Close']).quantize(q)
            row['Volume'] = int(row['Volume'])
            self.daily_historical_data[row['Date']] = row

        self.historical_data = self.daily_historical_data

## 算出および描画

In [7]:
for symbol in symbol_map:
    instrument = NewYahooSecurity(symbol, period)

    chart = PFChart(instrument, box_size=box_size, method=method, reversal=reversal)
    chart.create_chart()

    print(chart.chart)

[*********************100%***********************]  1 of 1 completed

  GOOGL  (2022-05-23 o: 2191.75 h: 2229.76 l: 2229.76 c: 2229.76)
  1.00% box, 3 box reversal, c method
  signal: sell status: bear confirmed

3016.49|                                                    |3016.49
2986.62|              x                                     |2986.62
2957.05|          x   u d x   x   x                         |2957.05
2927.78|          x B u d u d u 1 x d                       |2927.78
2898.79|  9       u d u d u d u d x d                       |2898.79
2870.09|  x d     u d   d C d u d x d                       |2870.09
2841.67|  x d x   u     o   d   d x o         x             |2841.67
2813.54|  x d x d u             d x o         x 4           |2813.54
2785.68|  x d x d u             d x o         x d           |2785.68
2758.10|  x o u d               d x o         x d           |2758.10
2730.79|  8 o u                 o 2 o         x o           |2730.79
2703.75|  x o A             